# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "demo_human_or_worm"
TOKENIZER = "kmer"
KMER = 3


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

demo_human_or_worm 0 kmer 3


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 71
{'NAG': 69, 'NNT': 66, 'NNN': 65, 'ACG': 62, 'GGC': 61, 'GGT': 60, 'CTG': 59, 'TCG': 56, 'CGC': 55, 'GCG': 54, 'GTG': 53, 'CCG': 50, 'CTC': 49, 'GAC': 57, 'CCC': 47, 'GCT': 44, 'CAA': 28, 'CCA': 42, 'GTC': 52, 'GCC': 31, 'CGA': 38, 'GAA': 40, 'ATG': 39, 'TCC': 34, 'AAC': 27, 'TTC': 24, 'AAG': 32, 'CAT': 23, 'TGC': 26, 'AGA': 13, 'CAC': 20, '<unk>': 0, 'GGG': 58, 'GAT': 2, 'ATT': 29, 'ACT': 6, 'TAA': 17, 'TCT': 16, 'CTA': 25, 'GTT': 15, 'TAC': 43, 'ATC': 46, 'TAT': 21, 'CGT': 48, 'GTA': 8, 'ATA': 1, 'ACA': 18, 'AAA': 11, 'GCA': 33, 'GAG': 63, 'TAG': 19, 'TGA': 12, 'CGG': 51, 'CTT': 10, 'TGG': 30, 'TTA': 9, 'TTG': 41, 'ACC': 36, 'GGA': 35, 'AAT': 7, 'ANN': 67, 'CCT': 5, 'TTT': 3, 'AGC': 45, '<pad>': 70, 'CAG': 64, 'TCA': 4, 'AGT': 22, 'AGG': 14, 'NNG': 68, 'TGT': 37}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  66


not all sequences are of the same length


In [9]:
tokenizer(train_dset[1][0])

['TTA',
 'AAC',
 'TTT',
 'AAA',
 'TTT',
 'TCC',
 'CAA',
 'ACT',
 'ACA',
 'GGA',
 'ACC',
 'CTT',
 'TGT',
 'TGA',
 'ATT',
 'TTA',
 'ACA',
 'GAT',
 'TTT',
 'TGA',
 'GCA',
 'CGA',
 'ATA',
 'CTA',
 'CAT',
 'TTT',
 'AAG',
 'ACA',
 'ATT',
 'TCC',
 'TCA',
 'TTC',
 'ATG',
 'TTA',
 'CTT',
 'CTT',
 'ATT',
 'TAA',
 'AGA',
 'AAT',
 'TTA',
 'TCA',
 'GAA',
 'ATT',
 'TAT',
 'TTA',
 'TTG',
 'AGG',
 'ACT',
 'ATG',
 'TAA',
 'TAA',
 'ACA',
 'TTT',
 'TTT',
 'AAT',
 'CAA',
 'CAT',
 'CTT',
 'TTT',
 'ATG',
 'AAC',
 'AAT',
 'TGA',
 'CCA',
 'TAG']

## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 88.2%, Avg loss: 0.561180 

Epoch 1


Train metrics: 
 Accuracy: 88.6%, Avg loss: 0.556692 

Epoch 2


Train metrics: 
 Accuracy: 89.1%, Avg loss: 0.555791 

Epoch 3


Train metrics: 
 Accuracy: 90.6%, Avg loss: 0.548647 

Epoch 4


Train metrics: 
 Accuracy: 90.6%, Avg loss: 0.546241 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  12500 ; tp  10796.045638561249 ; fp  909.520876654783
recall  0.8636836510848999 ; precision  0.9223001402390478
num_batches 782
correct 22396
size 25000
Test metrics: 
 Accuracy: 0.895840, F1 score: 0.892030, Avg loss: 0.550661 



(0.89584, 0.8920299908514573)